## Data Loading 

In [ ]:
import pandas as pd
import numpy as np
import difflib
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_sm
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import re
from functools import reduce
from word2number import w2n
import csv
import numpy as np
import os
import rdflib
import pandas as pd
from sklearn.metrics import pairwise_distances
from transformers import pipeline, set_seed


movie_info = pd.read_pickle(r'movies_new_9_12.pkl')
human_info = pd.read_pickle(r'humans_9_12.pkl')
crew_info = pd.read_pickle(r'movie_crew_9_12.pkl')
images_info  = pd.read_pickle(r'images_from_json.pkl')
movie_names_df = pd.read_pickle(r'list_of_movies_6_12.pkl')
human_names_df = pd.read_pickle(r'list_of_humans_6_12.pkl')
movie_multimedia = pd.read_pickle(r'movie_multimedia_6_12.pkl')
human_multimedia = pd.read_pickle(r'humans_multimedia_6_12.pkl')
entity_code_description_df = pd.read_pickle(r'entity_code_description_9_12.pkl')
all_property_code_description_df = pd.read_pickle(r'all_properties_mapping_9_12.pkl')
all_entities_code_description_df = pd.read_pickle(r'all_entities_mapping_10_12.pkl')
crowdsource_df = pd.read_pickle(r'crowdsource_df.pkl')
top_250_imdb_df = pd.read_pickle(r'top_250_imdb_df.pkl')
plots_df = pd.read_pickle(r'plots_df.pkl')
comments_df = pd.read_pickle(r'comments_df.pkl')
rating_codes_df = pd.read_pickle(r'rating_codes_df.pkl')
entity_id_df = pd.read_pickle(r'entity_ids_9_12.pkl')
relation_id_df = pd.read_pickle(r'relation_ids_9_12.pkl')
entity_code_description_df = pd.read_pickle(r'entity_code_description_9_12.pkl')

movie_names_list = list(set(movie_names_df['movie names'].tolist()))
human_names_list = list(set(human_names_df['human names'].tolist()))

# load the embeddings
entity_emb = np.load('entity_embeds.npy')
relation_emb = np.load('relation_embeds.npy')

#from transformers import BertTokenizer, BertModel # for tokenizing and sentiment analysis
ner_pipeline = pipeline('ner', model='dbmdz/bert-large-cased-finetuned-conll03-english')
# from transformers import pipeline
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
# cachedStopWords = nltk.download('stopwords')
cachedStopWords = stopwords.words("english")
spell = SpellChecker()
nlp=spacy.load("en_core_web_sm")

data_2={'property':['cast member','fsk film rating','imdb id','director','editor','nominated for',
'based on','box office','designer','citizenship', 'country of origin','subject', 'genre', 'screenwriter', 
 'occupation', 'photographer', 'publication date', 'production company']
,'code':['P161', 'P1981', 'P345', 'P57', 'P1040', 'P1411', 'P144', 'P2142', 'P2554', 'P27', 'P495',
 'P921', 'P136', 'P58', 'P106', 'P344', 'P577', 'P272']}

selective_propert_codes_df = pd.DataFrame(data_2)
# selective_propert_codes_df

### Helper lists for intent/properties

In [ ]:
list_of_MULTIMEDIA_words = ['pics', 'images', 'image', 'photo',' photos', 'photograph', 'photographs', 'portrait','portraits', 'still', 'stills', 'poster', 'posters','still frame', 'frame', 'frames', 'picture','pictures', 'look', 'looks']
list_of_RECOMMENDATION_words = ['recommend', 'similar', 'recommendations', 'recommendation', 'like']

list_of_occupation_words = ['occupation', 'for a living']
 
subcategory_multimedia = ['behind_the_scenes', 'still_frame', 'publicity', 'event', 'poster', 'production_art', 'product', 'user_avatar']

list_of_crew = ['director', 'screenwriter', 'cast member', 'editor', 'photographer', 'designer', 'production company']

list_of_movie_property = ['description','tags', 'publication date', 'genre',
 'rating','nominated','fsk rating','production company','based on'
,'cnc rating','imdb id','box office', 'country of origin','filmed location' ,'subject']

list_of_human_prop = ['description','occupation', 'language spoken', 'place of birth','citizenship','imdb id']

mapper_list_CAST_MEMBER = ['hero' ,'protagonist' ,'star' ,'starring' ,'heroine' ,'lead actor' ,'leading character' ,
'main character' , 'leading role', 'principal character' ,'Principal role' ,'star',
'chief character', 'chief participant' ,'leading participant' ,'main participant' ,'principal participant' ,
'central character' ,'key player', 'leading man' ,'leading figure' , 'superhero' ,
'leading performer' ,'leading actress' , 'celebrity', 'icon' ,'main figure']

greet_and_rem = ['hey', 'hi', 'hello', 'yo', 'yoo', 'film', 'movies', 'films', 'movie']

miscell_list = ['narrative', 'country', 'photography', 'mpaa','fsk','bbfc','rars','kijkwijzer','classind','incaa', 'kmrb','fpb','kavi','medierådet','ifco','nmhh','iroda','eirin','cnc(france)','icaa','cnc(romania)','jmk','rtc','igac','imda','hongkong','mtrcb','bamid'
'publication', 'detention' , 'death', 'birth' , 'place' , 'location' , 'burial', 'location' ,  'character', 'sound', 'art', 'executive', 'director'
'production', 'costume', 'designer'  ]

### Extract NER (excluding recommender questions)

In [ ]:
def get_ner(question):
    try:
        # question = question.title()
        ner = ner_pipeline(question, aggregation_strategy="simple")
        print(ner)
        if len(ner) == 0:
            print("INSIDE GET NER FUNCTION.... LENGTH IS 0")
            ner_output = ''
            
        else:
            scores = []
            for i in range (len(ner)):
                score = scores.append(ner[i]['score'])
            max_value = max(scores)
            index = scores.index(max_value)
            print("Index taken = " + str(index))

        ner_output = ner[index]
    except:
        ner_output = ''

    return ner_output

### Extract entities (Human + Movie)

In [ ]:
def human_movie_extractor(question):
    try:
        ner = get_ner(question)
        if len(ner) > 0:

            # print("***** SOMETHING IS DETECTED BY NER *****")
            entity_dict = ner
            if entity_dict['entity_group'] == 'MISC':
                mov_human_flag = 'movie'
                print("we perceive that the question related to the movie is asked ....")
                
                if entity_dict['word'] not in movie_names_list:
                    # print('*****************')
                    received_entity = entity_dict['word']
                    expected_movie = difflib.get_close_matches(entity_dict['word'], movie_names_list)[0]
                    usable_entity = expected_movie
                    # print("cannot find the movie. Trying with the closest movie that we have")
                    print("Closest movie found!! The expected movie is : " + expected_movie)
                    # print("usable entity : "+ usable_entity)
                    # print("receieved entity : "+ received_entity)
                    if len(expected_movie) == 0:
                        print("MOVIE NOT FOUND WITH NER")
                        received_entity = ''
                        usable_entity = ''

                else:
                    received_entity = entity_dict['word']
                    usable_entity = received_entity
                    print("BINGO!! EXCAT MOVIE IS GUESSED WITH NER")

            elif entity_dict['entity_group'] == 'PER':
                mov_human_flag = 'human'
                print("Question related to a PERSON is being asked ....")

                if entity_dict['word'] not in human_names_list:
                    received_entity = entity_dict['word']
                    expected_human = difflib.get_close_matches(entity_dict['word'], human_names_list)[0]
                    usable_entity = expected_human
                    # print("cannot find the person. Trying with the closest name of person that we have")
                    print("Closest person found!! The expected name is : " + expected_human)
                    # received_entity = expected_human ## remember to pass the original movie that the user entered to delete it from the question
                    if len(expected_human) == 0:
                        print("HUMAN NOT FOUND WITH NER")
                        received_entity = ''
                        usable_entity = ''
                else:
                    received_entity = entity_dict['word']
                    usable_entity = received_entity
                    # expected_entity = ''
                #print("the name of the person is : " + received_entity)

            else: ##GUESSing
                print("GUESSING SOME MOVIE/HUMAN USING NER")
                received_entity = entity_dict['word']
                expected_guess_1 = difflib.get_close_matches(entity_dict['word'], human_names_list)
                expected_guess_2 = difflib.get_close_matches(entity_dict['word'], movie_names_list)

                best_guess_1_score = 0
                best_guess_2_score = 0
                movie_found = ''
                human_found = ''
                
                if len(expected_guess_1) > 0:
                    best_guess_1 = expected_guess_1[0] 
                    best_guess_1_score = difflib.SequenceMatcher(None, entity_dict['word'], best_guess_1).ratio()
                    if best_guess_1 in human_names_list:
                        movie_found = 'yes'

                    print("score of entity being a human = " + str(best_guess_1_score))

                if len(expected_guess_2) > 0:
                    best_guess_2 = expected_guess_2[0] 
                    best_guess_2_score = difflib.SequenceMatcher(None, entity_dict['word'], best_guess_2).ratio()
                    if best_guess_2 in movie_names_list:
                        movie_found = 'yes'

                    print("score of entity being a movie = " + str(best_guess_2_score))


                if len(expected_guess_1) == 0 and len(expected_guess_2) == 0:
                    usable_entity = ''
                    mov_human_flag = ''
                    entity_code = ''
                    received_entity = ''

                    print("COULD NOT GUESS THE MOVIE/HUMAN USING NER *****")

                else :
                    if best_guess_1_score > best_guess_2_score and human_found == 'yes':
                        print("Guessing it to be the human from our list")
                        print("matching with : " + best_guess_1)
                        usable_entity = best_guess_1
                        mov_human_flag = 'human'

                    elif best_guess_1_score <=  best_guess_2_score and movie_found == 'yes':
                        print("Guessing it to be the movie from our list")
                        print("matching with : " + best_guess_2)
                        usable_entity = best_guess_2
                        mov_human_flag = 'movie'

            if len(entity_code_description_df[entity_code_description_df['description'] == usable_entity.lower()]['code']) > 0:
                entity_code = entity_code_description_df[entity_code_description_df['description'] == usable_entity.lower()]['code'].item()
            else :
                entity_code = ''

        if len(ner) == 0 or usable_entity == '' or entity_code == '' :
            print("ENTITY NOT FOUND USING NER")
            usable_entity, entity_code = fin_entity_matching(question)
            received_entity = usable_entity
            if len(usable_entity) == 0 :
                print("ENTITY NOT FOUND USING REGEXXX TOOO")
                usable_entity = '' 
                entity_code = ''
                mov_human_flag = ''
                received_entity = ''
                return mov_human_flag, received_entity, usable_entity, entity_code
            else:
                print("ENTITY FOUND USING REGEXXXXX " + str(usable_entity))
                if usable_entity in human_names_list:
                    mov_human_flag = 'human'
                else:
                    mov_human_flag = 'movie'
                
    except:
        mov_human_flag = ''
        received_entity = ''
        usable_entity = ''
        entity_code = ''

    return mov_human_flag, received_entity, usable_entity, entity_code

### extract n entities for recommender system 


In [ ]:
def human_movie_extractor_recommender(ner):
    try:
        print("***** INSIDE human_movie_extractor *****")
        # entity_dict = ner[0]
        entity_dict = ner
        if entity_dict['entity_group'] == 'MISC':
            mov_human_flag = 'movie'
            print("WE PERCEIVE THAT IT CAN BE A MOVIE Question ....")
            
            if entity_dict['word'] not in movie_names_list:
                print('*****************')
                received_entity = entity_dict['word']
                expected_movie = difflib.get_close_matches(entity_dict['word'], movie_names_list)[0]
                usable_entity = expected_movie
                print("cannot find the movie. Trying with the closest movie that we have")
                print("CLOSEST MATCH OF THE MOVIE IS FOUND =  " + expected_movie)
                # print("usable entity : "+ usable_entity)
                # print("receieved entity : "+ received_entity)
                # received_entity = expected_movie ## remember to pass the original movie that the user entered to delete it from the question
                if len(expected_movie) == 0:
                    print("CANNOT FIND THE MOVIE WITH THE NER")
                    received_entity = ''
                    usable_entity = ''
                    mov_human_flag = ''
            else:
                received_entity = entity_dict['word']
                usable_entity = received_entity
                print("WE HAVE RECEIVED THE EXACT MOVIE FROM NER = " + str(usable_entity))
                print("WILL EXTRACT THE CODE OF THE MOVIE NOW")
                # expected_entity = ''
            # print("Working on the movie : " + received_entity)

        elif entity_dict['entity_group'] == 'PER':
            mov_human_flag = 'human'
            print("Question related to a PERSON is being asked ....")

            if entity_dict['word'] not in human_names_list:
                received_entity = entity_dict['word']
                expected_human = difflib.get_close_matches(entity_dict['word'], human_names_list)[0]
                usable_entity = expected_human
                print("cannot find the person. Trying with the closest name of person that we have")
                print("Closest person found!! The expected name is : " + expected_human)
                # received_entity = expected_human ## remember to pass the original movie that the user entered to delete it from the question
                if len(expected_human) == 0:
                    print("CANNOT FIND THE HUMAN WITH THE NER")
                    received_entity = ''
                    usable_entity = ''
                    mov_human_flag = ''
            else:
                received_entity = entity_dict['word']
                usable_entity = received_entity
                print("WE HAVE RECEIVED THE EXACT HUMAN NAME FROM NER")
                # expected_entity = ''
            #print("the name of the person is : " + received_entity)

        else: ##GUESSing
            print("GUESSSING THE NAME OF THE MOVIE OR THE HUMAN FROM NER")
            received_entity = entity_dict['word']
            expected_guess_1 = difflib.get_close_matches(entity_dict['word'], human_names_list)
            expected_guess_2 = difflib.get_close_matches(entity_dict['word'], movie_names_list)

            best_guess_1_score = 0
            best_guess_2_score = 0
            movie_found = ''
            human_found = ''
            
            if len(expected_guess_1) > 0:
                best_guess_1 = expected_guess_1[0] 
                best_guess_1_score = difflib.SequenceMatcher(None, entity_dict['word'], best_guess_1).ratio()
                if best_guess_1 in human_names_list:
                    movie_found = 'yes'

                print("score of entity being a human = " + str(best_guess_1_score))

            if len(expected_guess_2) > 0:
                best_guess_2 = expected_guess_2[0] 
                best_guess_2_score = difflib.SequenceMatcher(None, entity_dict['word'], best_guess_2).ratio()
                if best_guess_2 in movie_names_list:
                    movie_found = 'yes'

                print("score of entity being a movie = " + str(best_guess_2_score))


            if len(expected_guess_1) == 0 and len(expected_guess_2) == 0:
                print("COULD NOT DECIDE IF THE ENTITY IS MOVIE OR HUMAN..... ---> GOING TO THE REGEXXXX")
                usable_entity = ''
                mov_human_flag = ''
                entity_code = ''
                received_entity = ''
                

            else :
                if best_guess_1_score > best_guess_2_score and human_found == 'yes':
                    print("Guessing it to be the human from our list")
                    print("matching with : " + best_guess_1)
                    usable_entity = best_guess_1
                    mov_human_flag = 'human'

                elif best_guess_1_score <=  best_guess_2_score and movie_found == 'yes':
                    print("Guessing it to be the movie from our list")
                    print("matching with : " + best_guess_2)
                    usable_entity = best_guess_2
                    mov_human_flag = 'movie'

        if len(entity_code_description_df[entity_code_description_df['description'] == usable_entity.lower()]['code']) > 0:
            entity_code = entity_code_description_df[entity_code_description_df['description'] == usable_entity.lower()]['code'][:1].item()
            print("YAYYY!! WE ALSO GOT THE CODE FOR THE ENTITY = " + str(usable_entity) + " The code  = " + str(entity_code))
            # print("ENTTY CODE FOR THE MOVIE " + usable_entity + " IS = " + str(entity_code))
            return mov_human_flag, received_entity, usable_entity, entity_code
        else :
            entity_code = ''
            print("DID NOT RECEIVE THE ENTITY CODE FROM THE NER. WILL LOOK FOR THE NEW ENTITY NAME USIGN REGEXXXX")

        if usable_entity == '' or entity_code == '' :
            print("ENTITY NOT FOUND USING NER.... TRYING WIHT REGEXXX NOW")
            usable_entity, entity_code = fin_entity_matching(question)
            received_entity = usable_entity
            if len(usable_entity) == 0 :
                print("ENTITY NOT FOUND USING REGEXXX TOOO")
                usable_entity = '' 
                entity_code = ''
                mov_human_flag = ''
                received_entity = ''
                return mov_human_flag, received_entity, usable_entity, entity_code
            else:
                print("ENTITY FOUND USING REGEXXXXX" + str(usable_entity))
                if usable_entity in human_names_list:
                    mov_human_flag = 'human'
                else:
                    mov_human_flag = 'movie'

    except:
        mov_human_flag = ''
        received_entity = ''
        usable_entity = ''
        entity_code = ''

    return mov_human_flag, received_entity, usable_entity, entity_code

### Removing the entity name from the question

In [ ]:
def entity_remove_from_question(ques, received_entity):
    ques = ques.title()
    # print("***** INSIDE entity_remove_from_question *****")
    rem_ques = ques.replace(received_entity, '')
    print('Remaining question after removing the movie name : ' + rem_ques)
    
    return rem_ques

### Removing stop words from the remaining question

In [ ]:
def stop_word_removal(ques, cachedStopWords):
    ques = ques.lower()
    rem_ques = ' '.join([word for word in ques.split() if word not in cachedStopWords])
    print('Remaining question after removing the stop words : ' + rem_ques)
    return rem_ques

### Property extractor --> 'movie'

In [ ]:
def property_extractor_movie(ques):
    print("***** INSIDE property_extractor_movie *****")
    for word in ques.split():

        # words.append(word)
        expected_actor = difflib.get_close_matches(word.lower(), mapper_list_CAST_MEMBER)
        # expected_actor_score = difflib.SequenceMatcher(None, word.lower(), expected_actor[0]).ratio()
        # print("nearest actor_entity : " + str(expected_actor))
        expected_prop_movie = difflib.get_close_matches(word.lower(), list_of_movie_property)
        # expected_prop_movie_score = difflib.SequenceMatcher(None, word.lower(), expected_prop_movie[0]).ratio()
        # print("nearest expected_prop_movie : " + str(expected_prop_movie))
        expected_crew_member = difflib.get_close_matches(word.lower(), list_of_crew)  
        # expected_crew_member_score = difflib.SequenceMatcher(None, word.lower(), expected_crew_member[0]).ratio()
        # print("nearest expected_crew_member : " + str(expected_crew_member))

        if len(expected_crew_member) > 0 :
            # expected_crew_member_score = difflib.SequenceMatcher(None, word.lower(), expected_crew_member[0]).ratio()
            print("For Movie, expected crew member is :" + expected_crew_member[0])
            print("original word is : " + word.lower())
            target_property = expected_crew_member[0]
            table_to_query = crew_info
            break

        if len(expected_actor) > 0 :#and expected_actor_score > expected_prop_movie_score:
            # expected_actor_score = difflib.SequenceMatcher(None, word.lower(), expected_actor[0]).ratio()
            print("For Movie, expected subset of cast member is :" + expected_actor[0])
            print("original word is : " + word.lower())
            target_property = 'cast member'
            table_to_query = crew_info
            break
        
        if len(expected_prop_movie) > 0 :
            # expected_prop_movie_score = difflib.SequenceMatcher(None, word.lower(), expected_prop_movie[0]).ratio()
            print("For Movie, No crew member property is asked")
            print("expected property is : " + expected_prop_movie[0])
            print("original word is : " + word.lower())
            target_property = expected_prop_movie[0]
            table_to_query = movie_info
            break

    try: target_property
    except NameError: target_property = None

    if target_property is None:
        target_property = ''
        table_to_query = ''
        property_code = ''
        return target_property, table_to_query, property_code

    elif len(selective_propert_codes_df[selective_propert_codes_df['property'] == target_property]['code']) > 0:
        property_code  = selective_propert_codes_df[selective_propert_codes_df['property'] == target_property]['code'].item()
    else :
        property_code = ''
        print("Property Code received is" + str(property_code))

    print('target property sent by property extractor = ' + target_property)
    return target_property, table_to_query, property_code

### Property extractor --> 'Human'

In [ ]:
def property_extractor_human(ques):
    print("***** INSIDE property_extractor_movie *****")
    for word in ques.split():
        expected_prop_human = difflib.get_close_matches(word.lower(), list_of_human_prop)
        print("nearest expected_prop_human : " + str(expected_prop_human))

        if word.lower() == 'who':
            print("Description of human")
            target_property = 'description'
            table_to_query = human_info
            break

        if len(expected_prop_human) > 0:
            print("expected property is : " + expected_prop_human[0])
            print("original word is : " + word.lower())
            target_property = expected_prop_human[0]
            table_to_query = human_info
            break

    try: target_property
    except NameError: target_property = None

    if target_property is None:
        target_property = ''
        table_to_query = ''
        property_code = ''
        return target_property, table_to_query, property_code

    elif len(selective_propert_codes_df[selective_propert_codes_df['property'] == target_property]['code']) > 0:
        property_code  = selective_propert_codes_df[selective_propert_codes_df['property'] == target_property]['code'].item()
    else :
        property_code = ''
        print("Property Code received is" + str(property_code))
    print('target property sent by human property extractor = ' + target_property)
    return target_property, table_to_query, property_code

### Factual Result generator --> 'Movie'

In [ ]:
def result_generator_movie(target_property, table_to_query, movie_name):

    print("***** INSIDE result_generator_movie *****")
    print("target property received by result generator is : " +target_property)
    print("movie name is : " + movie_name)

    if len(table_to_query[table_to_query['movie label'] == movie_name]) > 0:
        value_rows = table_to_query[table_to_query['movie label'] == movie_name]
        target_movie_info = list(set(value_rows[target_property].iloc[:1].item()))[0]
    else:
        target_movie_info = ''
        result_code = ''

    if len(entity_code_description_df[entity_code_description_df['description'] == target_movie_info.lower()]['code']) > 0 :
        result_code = entity_code_description_df[entity_code_description_df['description'] == target_movie_info.lower()]['code'].iloc[:1].item()
    else :
        result_code = ''

    if target_movie_info == 'None':
        target_movie_info = ''
        result_code = ''

    return target_movie_info, result_code


### Factual Result generator --> 'Human'

In [ ]:
def result_generator_human(target_property, table_to_query, human_name):
    print("target property received by result generatoris : " +target_property) 
    print("human name is : " + human_name)

    if len(table_to_query[table_to_query['human name'] == human_name]) > 0:
        value_rows = table_to_query[table_to_query['human name'] == human_name]
        target_human_info = list(set(value_rows[target_property].iloc[:1].item()))[0]

    else:
        target_human_info = ''
        result_code = ''

    if len(entity_code_description_df[entity_code_description_df['description'] == target_human_info.lower()]['code']) > 0 :
        result_code = entity_code_description_df[entity_code_description_df['description'] == target_human_info.lower()]['code'].iloc[:1].item()
    else :
        result_code = ''
    
    if target_human_info == 'None':
        target_human_info = ''
        result_code = ''

    return target_human_info  , result_code

### Universal property extractor


In [ ]:
def universal_property_extractor(rem_question):
    try:
        target_property= ''
        target_property_code = ''
        for word in rem_question.split():
            expected_property = difflib.get_close_matches(word.lower(), all_property_code_description_df.description.values.tolist())
            print("nearest UNIVERSAL PRPERTY expected property : " + str(expected_property))
            if len(expected_property) > 0:
                print("expected UNIVERSAL PRPERTY property is : " + expected_property[0])
                print("original UNIVERSAL PRPERTY word is : " + word.lower())
                target_property = expected_property[0]
                target_property_code = all_property_code_description_df[all_property_code_description_df['description'] == target_property]['code'].item()
                break
    except:
        target_property= ''
        target_property_code = ''
    return target_property, target_property_code



## 1. Get Factual Answers

In [ ]:
def get_factual_answers(question):
    usable_entity = '' 
    entity_code = ''
    target_property = ''
    prop_code = ''
    result_label = ''
    result_code = ''
    mov_human_flag = ''
    
    mov_human_flag,received_entity,usable_entity, entity_code = human_movie_extractor(question)

    if len(usable_entity) == 0 :
        print("ENTITY NOT FOUND  NEITHER WITH REGEXXX NOR WITH NER")
        usable_entity = '' 
        entity_code = ''
        target_property = ''
        prop_code = ''
        result_label = ''
        result_code = ''
        mov_human_flag = ''
        return usable_entity, entity_code, target_property, prop_code, result_label, result_code, mov_human_flag
    else:
        rem_ques = entity_remove_from_question(question, received_entity)
        miscell_flag = check_miscel_intents(question)
        if mov_human_flag == 'movie': # ONLY FOR FACTUAL
            
            if miscell_flag == 'comment':
                print('providing comments from movie answers')
                result_label = ''
                target_property = 'comment'
                result_code = ''
                prop_code = ''
                return usable_entity, entity_code, target_property, prop_code, result_label, result_code, mov_human_flag

            elif miscell_flag == 'plot':
                print('providing plot of movie answers')
                result_label = ''
                target_property = 'plot'
                result_code = ''
                prop_code = ''
                return usable_entity, entity_code, target_property, prop_code, result_label, result_code, mov_human_flag
  
            else:
                target_property, table_to_query_movie, prop_code = property_extractor_movie(rem_ques)
                if len(target_property) > 0:
                    result_label, result_code = result_generator_movie(target_property, table_to_query_movie, usable_entity)
                    print(result_label, result_code)
                    print('Entity Code of movie= ' + entity_code + " &&& Entity Name  of movie= " + usable_entity)
                    print('Property Code of movie = ' + prop_code + " &&& Property Name of Movie = " + target_property)
                    print('Result Code of movie = ' + result_code + " &&& Result Label of movie = " + result_label)
                else :
                    result_label = ''
                    result_code = ''

        elif mov_human_flag == 'human':
            target_property, table_to_query_human, prop_code = property_extractor_human(rem_ques)
            if len(target_property) > 0:
                result_label, result_code = result_generator_human(target_property, table_to_query_human, usable_entity)
                print(result_label, result_code)
                print('Entity Code  of human= ' + entity_code + " &&& Entity Name  of human = " + usable_entity)
                print('Property Code  of human= ' + prop_code + " &&& Property Name  of human = " + target_property)
                print('Result Code  of human= ' + result_code + " &&& Result Label  of human = " + result_label)
            else :
                result_label = ''
                result_code = ''
    
    return usable_entity, entity_code, target_property, prop_code, result_label, result_code, mov_human_flag

### Check factual answers with embedding answers

In [ ]:
# property_code = 'P1431'
# entity_code = 'Q223596'
# result_code = 'Q329737'
# result_label = 'butcher'
def get_factual_embedding_rank(property_code, entity_code, result_code, result_label):
    head = entity_emb[int(entity_id_df[entity_id_df['entity_id names'] == entity_code]['index'])]
    pred = relation_emb[int(relation_id_df[relation_id_df['relation_id names'] == property_code]['index'])]
    lhs = head + pred
    dist = pairwise_distances(lhs.reshape(1, -1), entity_emb).reshape(-1)
    most_likely = dist.argsort()
    ranks = dist.argsort().argsort()

    factual_rank = ranks[int(entity_id_df[entity_id_df['entity_id names'] == result_code]['index'])]

    for rank, idx in enumerate(most_likely[:1]):
        embedding_result_label = entity_code_description_df[entity_code_description_df['code'] == entity_id_df[entity_id_df['index'] == str(idx)]['entity_id names'].item()]['description'].item()
        embedding_rank = rank+1

    return factual_rank, embedding_rank, embedding_result_label

## 2. EMBEDDING ANSWERS :

In [ ]:
def get_embedding_result(property_code, entity_code):
    try:
        head = entity_emb[int(entity_id_df[entity_id_df['entity_id names'] == entity_code]['index'])]
        pred = relation_emb[int(relation_id_df[relation_id_df['relation_id names'] == property_code]['index'])]
        lhs = head + pred
        dist = pairwise_distances(lhs.reshape(1, -1), entity_emb).reshape(-1)
        most_likely = dist.argsort()
        ranks = dist.argsort().argsort()

        for rank, idx in enumerate(most_likely[:1]):
            embedding_result_label = entity_code_description_df[entity_code_description_df['code'] == entity_id_df[entity_id_df['index'] == str(idx)]['entity_id names'].item()]['description'].item()
            embedding_rank = rank+1
    except:
        embedding_result_label = ''
    return embedding_result_label

## 3. MULTIMEDIA ANSWERS :

In [ ]:
def get_image(ques):

    mov_human_flag, _, entity_name, _ = human_movie_extractor(ques)

    image_type = 'still_frame'
    target_property = 'imdb id'

    if mov_human_flag == 'human':

        table_to_query = human_info
        human_name = entity_name
        imdb_code = result_generator_human(target_property, table_to_query, human_name)
        imdb_code = list(imdb_code)
        mask = images_info.cast.apply(lambda x: any(item for item in imdb_code if item in x))
        possible_options = images_info[mask]
        # print(possible_options)
        # sel_option = possible_options[possible_options['type'] == image_type].iloc[0]  'Here is the image image:3543/rm1853225728'
        if len(possible_options[possible_options['type'] == image_type]) > 0:
            sel_option = possible_options[possible_options['type'] == image_type].iloc[0]
            image = sel_option['img'][:-4]
            render_image = "Here is the image of " + human_name +  " image:" + str(image)
            print("Image URL is : " + render_image)
        elif len(possible_options) > 0:
            sel_option = possible_options.iloc[0]
            image = sel_option['img'][:-4]
            render_image = "Here is the image of " + human_name +  " image:" + str(image)
            print("Image URL is : " + render_image)
        else :
            render_image = 'Dear human, I was unable to find the required image of ' + entity_name

    elif mov_human_flag == 'movie':

        # print('$$$$$$$$$$$$$$$$$$$$$')
        table_to_query = movie_info
        movie_name = entity_name
        imdb_code = result_generator_movie(target_property, table_to_query, movie_name)
        imdb_code = list(imdb_code)
        # print(str(imdb_code) + 'sdfsdfsdgsdgsdgsdgsdg')
        mask = images_info.movie.apply(lambda x: any(item for item in imdb_code if item in x))
        possible_options = images_info[mask]
        # print(possible_options)
        # sel_option = possible_options[possible_options['type'] == image_type].iloc[0]
        if len(possible_options[possible_options['type'] == image_type]) > 0:
            sel_option = possible_options[possible_options['type'] == image_type].iloc[0]
            image = sel_option['img'][:-4]
            render_image = "Here is the image from " + movie_name +  " image:" + str(image)
            print("Image URL is : " + render_image)
        elif len(possible_options) > 0:
            sel_option = possible_options.iloc[0]
            image = sel_option['img'][:-4]
            render_image = "Here is the image from " + movie_name +  " image:" + str(image)
            print("Image URL is : " + render_image)
        else :
            render_image = 'Dear human, I was unable to find the desired image from the movie : ' + entity_name
    
    else :
        render_image = 'Dear human, I was unable to find the desired image. Please give me another chance with a different entity'
    
    return render_image



## 4. RECOMMENDATION :

In [ ]:
def get_n_entites_recommender(question):
    # question = question.title()
    try:
        ner = ner_pipeline(question, aggregation_strategy="simple")
        print(ner)
        if len(ner) == 0:
            # print("Sorry, I did not understand the question")
            entity_name_list = []
            entity_code_list = []
        else:
            entity_name_list = []
            entity_code_list = []
            for i in range (len(ner)):
                _, _, usable_entity, entity_code = human_movie_extractor_recommender(ner[i]) 
                print("entity name = " + usable_entity + "entity code = " + entity_code)
                if len(entity_code) > 0 :
                    entity_name_list.append(usable_entity)
                    entity_code_list.append(entity_code)
        print("SENDING THE ENTITY LIST AND LABELS")
        print("ENTIT LABELS SENT TO RECOMMENDER SYSTEM  = " + str(entity_name_list))
        print("ENTIT CODE  SENT TO RECOMMENDER SYSTEM  = " + str(entity_code_list))
    except:
        print("COULD NOT IDENTIFY ANY ENTITIES SO THE RESULT WOULD BE AN EMPTY LIST")
        entity_name_list = []
        entity_code_list = []
    return entity_name_list, entity_code_list

In [ ]:
def get_recommendations_n_entity(question):
    try:
        list_entity_labels, list_entity_codes = get_n_entites_recommender(question)
        print("GOT THE ENTITY LISTS AND LABELS FOR THE RECOMMENDATIONS")
        list_entity_labels = list(map(str.lower,list_entity_labels))
        num_of_recommendations = len(list_entity_codes) + 5
        s_entity_emb = np.zeros(shape = (256,))
        for entity_code in list_entity_codes:
            # print(entity_code)
            s_entity_emb = s_entity_emb + entity_emb[int(entity_id_df[entity_id_df['entity_id names'] == entity_code]['index'])]

        # avg_entity_emb = s_entity_emb/len(list_entity_codes)

        dist = pairwise_distances(s_entity_emb.reshape(1, -1) , entity_emb).reshape(-1)
        most_likely = dist.argsort()

        embedding_result_label = []
        for rank, idx in enumerate(most_likely[:num_of_recommendations]):
            entity_codes_embed = entity_id_df[entity_id_df['index'] == str(idx)]['entity_id names'].item()
            # print(entity_codes_embed)
            if len(entity_code_description_df[entity_code_description_df['code'] == entity_codes_embed]['description']) == 0:
                continue
            else :
                entity_labels_embed = entity_code_description_df[entity_code_description_df['code'] == entity_codes_embed]['description']
                # print(entity_labels_embed)
                embedding_result_label.append(entity_labels_embed.item())
            # embedding_result_label.append(entity_code_description_df[entity_code_description_df['code'] == entity_id_df[entity_id_df['index'] == str(idx)]['entity_id names'].item()]['description'].item())
            # embedding_rank = rank+1
        result = [x for x in embedding_result_label if x not in list_entity_labels]
        if len(result) > 0:
            result_text = ', '.join([str(x).title() for x in result])
            result = "Based on your interests I can recommend you : " +  result_text
        else :
            result_text = ', '.join([str(x).title() for x in list_entity_labels])
            result = "Based on your interests I can recommend you : " +  result_text

    except:
        result_text = ', '.join([str(x).title() for x in list_entity_labels])
        result = "Based on your interests I can recommend you : " +  result_text
    return result

## 5. CROWDSOURCING :

In [ ]:
def get_crowd_source_data(entity_code, property_code):
    # if len(crowdsource_df[crowdsource_df['Input1ID'] == entity_code]) > 0 and len(crowdsource_df[crowdsource_df['Input2ID'] == property_code]) > 0:
    if len(crowdsource_df[(crowdsource_df['Input2ID'] == property_code) & (crowdsource_df['Input1ID'] == entity_code)]) == 1 : 
        # print('#####')
        correct_labeled = crowdsource_df[(crowdsource_df['Input2ID'] == property_code) & (crowdsource_df['Input1ID'] == entity_code)]['CORRECT'].item()
        incorrect_labeled =  crowdsource_df[(crowdsource_df['Input2ID'] == property_code) & (crowdsource_df['Input1ID'] == entity_code)]['INCORRECT'].item()
        result = crowdsource_df[(crowdsource_df['Input2ID'] == property_code) & (crowdsource_df['Input1ID'] == entity_code)]['Input3ID'].item()
        print(result)

        if result.startswith('P'):
            if len(all_property_code_description_df[all_property_code_description_df['code'] == result]['description']) > 0:
                result_label  = all_property_code_description_df[all_property_code_description_df['code'] == result]['description'].item()
            else:
                result_label = '' 
        elif result.startswith('Q'):
            if len(all_entities_code_description_df[all_entities_code_description_df['code'] == result]['description']) > 0:
                result_label  = all_entities_code_description_df[all_entities_code_description_df['code'] == result]['description'].item()
            else:
                result_label = '' 
        else :
            result_label = result

    else:
        result_label= ''
        correct_labeled = ''
        incorrect_labeled = ''

    return result_label, correct_labeled, incorrect_labeled


# get_crowd_source_data('Q1032889', 'P58')

### Intent Identification : 

In [ ]:
def rating_intent_get_property(ques):
    ques = ques.lower()
    property_label = ''
    property_code = ''
    for word in ques.split():
        # print(word)
        if word in rating_codes_df['description'].values.tolist():
            property_label = word
            property_code = rating_codes_df[rating_codes_df['description'] == word]['code'].item()
            return property_label, property_code
    if len(property_code) == 0:
        property_label = 'FSK film rating'
        property_code = 'P1981'
        return property_label, property_code

    return property_label, property_code

def location_intent_get_property(ques):
    ques = ques.lower()
    rem_stop_ques = stop_word_removal(ques, cachedStopWords)
    property_label = ''
    property_code = ''
    for word in rem_stop_ques.split():
        word = spell.correction(word)
        if word in ['work', 'working', 'works', 'worked']:
            property_label = 'work location'
            property_code = 'P937'
            break
        elif word in ['narrative', 'narative', 'narration', 'narrated', 'narated', 'narrating', 'narating', 'narrates', 'narates', 'narrate', 'narate']:
            property_label = 'narrative location'
            property_code = 'P840'
            break
        elif word in ['origin']:
            property_label = 'country of origin'
            property_code = 'P495'
            break
        elif word in ['citizen', 'citizenship', 'passport', 'nationality']:
            property_label = 'country of citizenship'
            property_code = 'P27'
            break
        elif word in ['publication', 'published']:
            property_label = 'place of publication'
            property_code = 'P291'
            break
        elif word in ['detention', 'detained']:
            property_label = 'place of detention'
            property_code = 'P2632'
            break
        elif word in ['death']:
            property_label = 'place of death'
            property_code = 'P20'
            break
        elif word in ['birth', 'born', 'birthplace']:
            property_label = 'place of birth'
            property_code = 'P19'
            break
        elif word in ['burial', 'buried', 'burried']:
            property_label = 'place of burial'
            property_code = 'P119'
            break
            
    return property_label, property_code

def designer_intent_get_property(ques):
    ques = ques.lower()
    rem_stop_ques = stop_word_removal(ques, cachedStopWords)
    property_label = ''
    property_code = ''
    for word in rem_stop_ques.split():
        word = spell.correction(word)
        if word in ['character', 'characters']:
            property_label = 'character designer'
            property_code = 'P8670'
            break
        elif word in ['sound', 'sounds']:
            property_label = 'sound designer'
            property_code = 'P5028'
            break
        elif word in ['production', 'productions']:
            property_label = 'production designer'
            property_code = 'P2554'
            break
        elif word in ['costumes', 'costume']:
            property_label = 'costume designer'
            property_code = 'P2515'
            break
    return property_label, property_code

def director_intent_get_property(ques):
    ques = ques.lower()
    rem_stop_ques = stop_word_removal(ques, cachedStopWords)
    property_label = ''
    property_code = ''
    for word in rem_stop_ques.split():
        word = spell.correction(word)
        print(word)
        if word in ['art', 'arts']:
            property_label = 'art director'
            property_code = 'P3174'
            break
        elif word in ['photography', 'sounds']:
            property_label = 'director of photography'
            property_code = 'P344'
            break
    if len(property_code) == 0:     
        property_label = 'director'
        property_code = 'P57'
    return property_label, property_code

def check_image_recommendation(ques):
    rem_stop_ques = stop_word_removal(ques, cachedStopWords)
    intent_word_image = ''
    intent = ''
    for word in rem_stop_ques.split():
        word = spell.correction(word)
        if word in list_of_MULTIMEDIA_words:
            print("Image question is asked")
            intent = 'image'
            intent_word_image = word
            break
        elif word in list_of_RECOMMENDATION_words:
            print("Recommendation question asked")
            intent = 'recommendation'
            break
    return intent, intent_word_image

def get_property_from_intents(ques):
    # rem_stop_ques = stop_word_removal(ques, cachedStopWords)
    ques = ques.lower()
    property_label = ''
    property_code = ''
    for word in ques.split():
        word = spell.correction(word)
        # print(word)
        if word in ['designer', 'designed', 'designing']:    
            print('designer_intent_get_property')    
            property_label, property_code = designer_intent_get_property(ques)
            return property_label, property_code
        elif word in ['rating', 'rated', 'rate', 'ratings', 'rates']:
            print('rating_intent_get_property')  
            property_label, property_code = rating_intent_get_property(ques)
            return property_label, property_code
        elif word in ['where', 'country', 'location', 'located','place', 'nationality', 'passport', 'citizenship', 'citizen']:
            print('location_intent_get_property')  
            property_label, property_code = location_intent_get_property(ques)
            return property_label, property_code
        elif word in ['director', 'directed']:
            property_label, property_code = director_intent_get_property(ques) 
            return property_label, property_code
        elif word in ['living', 'occupation']:
            print('occupation is the PROPERTY')
            property_label = 'occupation'
            property_code = 'P106'
            return property_label, property_code
        elif word in ['executive', 'producer']:
            print('executive producer is the property')
            property_label = 'executive producer'
            property_code = 'P1431'
        elif word in ['actor', 'acted', 'acts', 'actress', 'hero', 'heroine']:
            print('cast member is the property')
            property_label = 'cast member'
            property_code = 'P161'
            return property_label, property_code
         
        # elif word in ['Zizou']:
        #     intent = 'Tom Meets Zizou'
    return property_label, property_code
        
def check_miscel_intents(ques):
    ques = ques.lower()
    count = 0
    misc_flag = ''
    for word in ques.split():
        # word = spell.correction(word)
        if word in ['comment', 'comments']:
            misc_flag = 'comment'
            break
        elif word in ['plot', 'plots']:
            misc_flag = 'plot'
            break
        elif word in ['imdb', 'top', 'best', 'greatest']:
            count = count +1 
            if count > 1:
                misc_flag = 'imdb' 
                break

    return misc_flag 

def miscell_comments(entity_name): ## should be a movie
    try :
        entity_name = entity_name.lower()
        answer = comments_df[comments_df['movie label'] == entity_name]['comment'].tolist()
        first_comment = answer[0]
        comment_to_string = ', '.join([str(x) for x in first_comment])
        answer = comment_to_string[:600]
    except:
        answer = ''
    return answer

def miscell_plot(entity_code): ## should be a movie
    try :
        plot = plots_df[plots_df['qid'] == entity_code]['plot'].tolist()
        plot_first = plot[0]
        plot_to_string = ', '.join([str(x) for x in plot])
        answer = plot_to_string[:600]
        return answer 
        
    except:
        answer = ''
    return answer 

def miscell_imdb(ques):
    try:
        print("inside try")
        number = w2n.word_to_num(ques)
        print(number)
    except:
        if len(re.findall(r'\d+', ques)) == 0:
            number = 5
        else :
            number = re.findall(r'\d+', ques)
            number = int(number[0])
        print("inside exception")
    print("going to answer")
    print(number)
    if number > 10:
        number = 10
    answer = top_250_imdb_df[:number]['movie label'].tolist()
    list_movie = ', '.join(str(item) for item in answer)
    answer_label = "Here are the top " + str(number) + " imdb movies of all time: " + list_movie
    return answer_label


In [ ]:
# p, c = get_property_from_intents('what is the rating of Top Gun?')
# print(p)
# print(c)

### REGEXX matching for the movies/human : 

In [ ]:
def preprocess_question(ques):
    try:

        ques = ques.lower()
        text_tokens = word_tokenize(ques)
        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
        print('tokenizer = ' + str(tokens_without_sw))
        new_list = [
            re.sub(r'[^a-zA-Z0-9 ]+', '', item) for item in tokens_without_sw
        ]
        new_list = list(filter(None, new_list))
        print(new_list)
        clean = [
            reduce(lambda item,loc: item.replace(loc,''), [item]+greet_and_rem+list_of_crew+list_of_movie_property+list_of_human_prop+list_of_MULTIMEDIA_words+list_of_RECOMMENDATION_words+miscell_list)
            for item in new_list
        ]
        clean = list(filter(None, clean))
    except:
        clean = []
    return clean

def fin_entity_matching(ques):
    try:
        tokens = preprocess_question(ques)
        for word in tokens:
            test_ans = all_entities_code_description_df.loc[all_entities_code_description_df['description'].str.contains(word, case=False)]
            if len(test_ans)>0:
                entity_name = test_ans[:1]['description'].item()
                entity_code = test_ans[:1]['code'].item()
                break
    except:
        entity_name = ''
        entity_code = ''
    return entity_name, entity_code
    

## Factual KG + CS + Embeddings:

In [ ]:
def get_factual_embed_cs_answers(question):

    try:
        
        entity_name, entity_code, property_name, property_code, result_name, result_code, mov_human_flag = get_factual_answers(question)

        if property_name == 'comment':
            ans = miscell_comments(entity_name)
            # print(ans)
            if len(ans) == 0:
                ans_message = "Dang!! We do not have the user comments for the movie " + entity_name 
            else:
                ans_message = "Here are the first few characters from one of the comment : " + entity_name + " : " + str(ans)
            return ans_message
        elif property_name == 'plot':
            ans = miscell_plot(entity_code)
            # print(ans)
            if len(ans) == 0:
                # print('ENTITY CODE = ' + entity_code)
                ans_message = "Dang!! We do not have the movie plot for  " + entity_name 
            else:
                ans_message = "Here are the first few characters of the plot for the movie " + entity_name + " : " + str(ans)
            return ans_message

            
        property_name_override, property_code_override = get_property_from_intents(question) ## OVERRIDE PROPERTIES
        if len(property_name_override) > 0:
            property_name = property_name_override
            property_code = property_code_override

        if len(entity_code) > 0 and len(property_code) > 0 and len(result_code) > 0: # KG + EMB
            print(" 1st CASE :: KG + EMBEDDINGS")
            facual_rank, embedding_rank, embedding_result = get_factual_embedding_rank(property_code, entity_code, result_code, result_name)
            if embedding_result == result_name.lower():
                answer = embedding_result
                ans_message = "Hey! The " + property_name + " of the " + entity_name + " is : " + str(answer.title()) + ". I got this answer verified from the graph as well as the embeddings"
            else:
                ans_message = "Hey!! I have received the " + property_name + " of the " + entity_name + " as " + str(result_name.title()) + " From the KG. But I have received " + str(embedding_result.title()) + " as the answer from the embeddings"
            return ans_message
        elif len(result_code) == 0 and len(result_name) > 0: # KG + CS
            print(" 2nd CASE :: KG + CS")
            print("entity name " + entity_name)
            print("property name " + property_name)
            print("entity code " + entity_code)
            print("property code " + property_code)
            ans_message = "Hey! I found the answer from the graph!! The " + property_name + " of the " + entity_name + " is : " + result_name
            result_label_cs, correct_labeled_cs, incorrect_labeled_cs = get_crowd_source_data(entity_code, property_code)
            if len(result_label_cs) > 0:
                print("RECEIVED CROWSOURCED DATA")
                ans_message = str(ans_message) + " Whereas, the " + str(property_name) +   " of " + str(entity_name) + " according to the crowd is " + str(result_label_cs.title()) + " The crowd had an inter rater agreement of 0.72 in this batch. The result distribution for this task was " + str(correct_labeled_cs) + " support votes and " + str(incorrect_labeled_cs) + " reject votes."
            return ans_message
        elif len(result_name) == 0 and len(entity_code) > 0 and len(property_code) > 0 : # CS + EMB
            print(" 3rd CASE :: CS + EMBEDDINGS")
            ans_message = ''
            result_label_cs, correct_labeled_cs, incorrect_labeled_cs = get_crowd_source_data(entity_code, property_code)
            embedding_result2 = get_embedding_result(property_code, entity_code)
            if len(embedding_result2) > 0:
                ans_message = "According to the embeddings, " + property_name + " of " + entity_name + " is " + str(embedding_result2.title())

            if len(result_label_cs) > 0:
                print("RECEIVED CROWSOURCED DATA")
                ans_message = str(ans_message) + " Whereas, I also have the result from the crowd as : " + str(result_label_cs.title()) + " The crowd had an inter rater agreement of 0.72 in this batch. The result distribution for this task was " + str(correct_labeled_cs) + " support votes and " + str(incorrect_labeled_cs) + " reject votes."

            if len(embedding_result2) == 0 and len(result_label_cs) == 0:
                ans_message = "Hey Hooman!! You got me this time. I did not find the answer to this question. I checked the KG, the embeddings and the crowdsourced data. Please give me another chance with a different question "
            return ans_message

        elif len(entity_code) > 0 and len(property_code) == 0 : 
            print(" 4th CASE :: Description")
            print("NO PROPERTY DETECTED. ONLY THE ENTITY NAME IS DETECTED. RETURNING THE DESCRIPTION and IMAGE")
            if mov_human_flag == 'movie':
                table_to_query = movie_info
                target_property = 'description'
                description , _ = result_generator_movie(target_property, table_to_query , entity_name)
                ans_message = "I am very sorry that i did not understand the property of " + entity_name + " , which is a " + description 
            elif mov_human_flag == 'human':
                table_to_query = human_info
                target_property = 'description'
                description , _ = result_generator_human(target_property, table_to_query, entity_name)
                ans_message = "I am very sorry that i did not understand the property of " + entity_name + " , who is a " + description
            return ans_message
            
        elif len(entity_code) == 0 and len(property_code) > 0:                # HERE WE ONLY HAVE THE PROPERTY
            print(" 5th CASE :: Random answer for the property")
            embedding_result3 = get_embedding_result(property_code, 'Q110138')
            if len(embedding_result3) == 0:
                embedding_result4 = get_embedding_result(property_code, 'Q37079')
                if len(embedding_result4) == 0:
                    ans_message = "soo.. I believe the " + property_name + " is : William"
                else:
                    ans_message = "voila!! The " + property_name + " is : " + str(embedding_result4)
            else:
                ans_message = "voila!! The " + property_name + " is : " + str(embedding_result3)
            return ans_message

        else:
            ans_message = "Daammnnn!! That was a tough one!! Can you please try to capitalise the first letter of the movie names and the entity name or try with an easier property, please ;)"
            return ans_message

    except:
        ans_message = "Wooff!! I did not see that coming!! Can you please try to titalise the entity name or try with an easier property, please ;)"
    return ans_message       

## ENDGAME 

In [ ]:
# question = 'show me the image of johhnny depp'
# question = 'What does Rohit Shetty look like?'
# question = 'Recommend movies like Nightmare on Elm Street, Friday the 13th, and Halloween.'
# question = 'show me some comments from teh movie Cliffhanger'
question = 'What is the place of burial of Sushant Singh Rajput'
question = 'What is the place of death of Sushant Singh Rajput'
question = 'What is the place of death of Shahrukh Khan'
question = 'Who is Shah Rukh Khan'
question = 'show me a still of Shah Rukh Khan'
question = 'who is the costume designer of Shape of Water'
# question = 'who is the art director of Shape of Water'
# question = 'who is the prodcution designer of Shape of Water'
# question = 'what is the narrative location of Dangal'
# question = 'who is teh actor in Dangal'
# question = 'Recommend movies like Nightmare on Elm Street, Friday the 13th, and Halloween.'
# question = 'Who is the executive producer of X-Men: First Class?'
# question = 'Can you tell me the publication date of Tom Meets Zizou?'
# question = 'What is the box office of The Princess and the Frog?'
# question = 'Given that I like The Lion King, Po cahontas, and The Beauty and the Beast, can you recommend some movies?'
# question = 'Recommend movies similar to Hamlet and  Othello.'
# question = 'Recommend movies like Nightmare on Elm Street, Friday the 13th, and Halloween.'
# question = 'Who is the executive 
# producer of Naruto the Movie: Guardians of the Crescent Moon Kingdom?'
# question = 'who is the screenwriter of Free Willy 3: The Rescue?' #**
# question = 'occupation of Jean Van Hamme' 
question = 'What is the MPAA film rating of Weathering with You?'
# question = 'who is the executive producer of TOP Gun' # **
# question = 'who is the director  of TOP Gun' # **
# question = 'Who is the executive producer of Naruto the Movie: Guardians of the Crescent Moon Kingdom?'
# question = 'Who is the screenwriter of Free Willy 3: The Rescue ?'
# question = 'Who is the executive producer of Naruto the Movie?'
# question = 'Who is the screenwriter of The Masked Gang: Cyprus?'
# question = 'What is the genre of Good Neighbors?'
# question = 'Who is the director of Star Wars: Episode VI - Return of the Jedi?'
# question = 'Who directed The Bridge on the River Kwai?'
# question = 'Who is the director of Good Will Hunting?'
# question = 'who is the screenwriter of Free Willy 3: The Rescue?'
# question = 'occupation of Jean Van Hamme'
# question = 'who is the executive producer of TOP Gun '
# question = 'Who is the executive producer of Naruto the Movie: Guardians of the Crescent Moon Kingdom?'
# question = 'Who is the screenwriter of Free Willy 3: The Rescue ?'
# question = 'Show me a picture of Halle Berry.'
# ques = 'hey! can you tell me the director of the movie Naruto the Movie: Guardians of the Crescent Moon Kingdom'
# ques = 'What is the MPAA film rating of Weathering with You?'
# # ques = 'Recommend movies similar to Hamlet and Othello.'
# ques = 'What is the MPAA film rating of Weathering with You?'

def get_answers_prod(question):
    try:
        if len(question) < 15:
            candidate_labels = ['positive sentiment', 'negative sentiment', 'greeting', 'neutral']
            question_intent = zero_shot_classifier(question, candidate_labels)
            user_intent = question_intent['labels'][0]
            if user_intent == 'positive sentiment':
                answer = "I am glad that you liked the answer. Try out some more questions :D"
            elif user_intent == 'negative sentiment':
                answer = "I am soo sorry you did not like my answer. I will try my best in the next one :D"
            elif user_intent == 'greeting':
                answer = "Hello dear human!! I hope you are well. Let's start the QnA round to test me out :D"
            elif user_intent == 'neutral':
                answer = "Try some more questions.. :)"
            return answer

        flag = check_miscel_intents(question)
        if flag == 'imdb':
            answer = miscell_imdb(question)
            return answer
        intent, intent_word_image = check_image_recommendation (question)
        if intent == 'image':
            print("IMAGE QUESTION")
            answer = get_image(question)
        elif intent == 'recommendation':
            print("RECOMMENDATION QUESTION")
            answer =  get_recommendations_n_entity(question)
        else :
            print("KG EMBEDDING CS QUESTION")
            answer = get_factual_embed_cs_answers(question)
            return answer
        
    except:
        answer = "Oops! I got some unprecedented error. Please ask me another question. Thanks :)"
    return answer

print(get_answers_prod(question))

## CONNECTION TO THE API

In [ ]:
import time
import atexit
import getpass
import requests  # install the package via "pip install requests"
from collections import defaultdict

# url of the speakeasy server
url = 'https://speakeasy.ifi.uzh.ch'
listen_freq = 3


class DemoBot:
    def __init__(self, username, password):
        self.agent_details = self.login(username, password)
        self.session_token = self.agent_details['sessionToken']
        self.chat_state = defaultdict(lambda: {'messages': defaultdict(dict), 'initiated': False, 'my_alias': None})

        atexit.register(self.logout)

    def listen(self):
        while True:
            # check for all chatrooms
            current_rooms = self.check_rooms(session_token=self.session_token)['rooms']
            for room in current_rooms:
                # ignore finished conversations
                if room['remainingTime'] > 0:
                    room_id = room['uid']
                    if not self.chat_state[room_id]['initiated']:
                        # send a welcome message and get the alias of the agent in the chatroom
                        self.post_message(room_id=room_id, session_token=self.session_token, message=f'Hello dear human, I am your movie wikipedia. You can ask me 1) factual questions related to the directors, screenwriters, ratings, date of publication of the movies. 2) I can answer questions related to the place of birth, place of death, occupation of the humans. 3) images from the movies and portraits of film actor/actress. 4) I can recommend you movies similar to what you like. 5) Plots and user comments for the movies. 6) list of top imdb movies. 7) Replies to your greetings and remarks. PS - Please note that the proper nouns (For ex. name of the person or a movie should start with a capital letter :) Thanks for your understanding. Also, you can write "good, nice, etc" to prasie the answer if you like and can criticise it as well. lets get started :)')
                        self.chat_state[room_id]['initiated'] = True
                        self.chat_state[room_id]['my_alias'] = room['alias'] # ******* CHECK THIS LATER *******

                    # check for all messages
                    all_messages = self.check_room_state(room_id=room_id, since=0, session_token=self.session_token)['messages']

                    # you can also use ["reactions"] to get the reactions of the messages: STAR, THUMBS_UP, THUMBS_DOWN

                    for message in all_messages:
                        if message['authorAlias'] != self.chat_state[room_id]['my_alias']:

                            # check if the message is new
                            if message['ordinal'] not in self.chat_state[room_id]['messages']:
                                self.chat_state[room_id]['messages'][message['ordinal']] = message
                                print('\t- Chatroom {} - new message #{}: \'{}\' - {}'.format(room_id, message['ordinal'], message['message'], self.get_time()))

                                ##### You should call your agent here and get the response message #####
                                # response = full_function(message['message'], room_id).encode('utf-8') # resp
                                # response = 'Here is the image image:3543/rm1853225728'
                                response = get_answers_prod(message['message'])
                                self.post_message(room_id=room_id, session_token=self.session_token, message = response)
                                # self.post_message(room_id=room_id, session_token=self.session_token, message='Got your message: \'{}\' at {}.'.format(message['message'], self.get_time()))
            time.sleep(1) ##

    def login(self, username: str, password: str):
        agent_details = requests.post(url=url + "/api/login", json={"username": username, "password": password}).json()
        print('- User {} successfully logged in with session \'{}\'!'.format(agent_details['userDetails']['username'], agent_details['sessionToken']))
        return agent_details

    def check_rooms(self, session_token: str):
        return requests.get(url=url + "/api/rooms", params={"session": session_token}).json()

    def check_room_state(self, room_id: str, since: int, session_token: str):
        return requests.get(url=url + "/api/room/{}/{}".format(room_id, since), params={"roomId": room_id, "since": since, "session": session_token}).json()

    def post_message(self, room_id: str, session_token: str, message: str):
        tmp_des = requests.post(url=url + "/api/room/{}".format(room_id),
                                params={"roomId": room_id, "session": session_token}, data=message).json()
        if tmp_des['description'] != 'Message received':
            print('\t\t Error: failed to post message: {}'.format(message))

    def get_time(self):
        return time.strftime("%H:%M:%S, %d-%m-%Y", time.localtime())

    def logout(self):
        if requests.get(url=url + "/api/logout", params={"session": self.session_token}).json()['description'] == 'Logged out':
            print('- Session \'{}\' successfully logged out!'.format(self.session_token))


if __name__ == '__main__':
    username = 'kartikey.sharma_bot'
    password = getpass.getpass('Password of the demo bot:')
    demobot = DemoBot(username, password)
    demobot.listen()


